## Gather Data

In [264]:
import requests, io
import pandas as pd
import zipfile
from gurobipy import Model, GRB, quicksum
from IPython.display import clear_output
from collections import defaultdict
import math

In [265]:


ridership_df = pd.DataFrame()

links = [
    'https://www.bart.gov/sites/default/files/2025-11/Ridership_202510.xlsx',
    'https://www.bart.gov/sites/default/files/2025-10/Ridership_202509.xlsx',
    'https://www.bart.gov/sites/default/files/2025-10/Ridership_202508.xlsx',
    'https://www.bart.gov/sites/default/files/2025-08/Ridership_202507.xlsx',
    'https://www.bart.gov/sites/default/files/2025-07/Ridership_202506.xlsx',
    'https://www.bart.gov/sites/default/files/2025-06/Ridership_202505.xlsx',
    'https://www.bart.gov/sites/default/files/2025-05/Ridership_202504.xlsx',
    'https://www.bart.gov/sites/default/files/2025-05/Ridership_202503.xlsx',
    'https://www.bart.gov/sites/default/files/2025-03/Ridership_202502.xlsx',
    'https://www.bart.gov/sites/default/files/2025-02/Ridership_202501.xlsx'
]

def extract_date_from_link(link):
    """
    Extract substring between 'Ridership_' and '.xlsx'.
    Example: 'Ridership_202510.xlsx' → '202510'
    """
    start = link.find("Ridership_") + len("Ridership_")
    end = link.find(".xlsx")
    return link[start:end]

def set_header_row(df, keyword="Exit Station Two-Letter Code"):
    """
    Find the row containing keyword and use it as the column headers.
    Return the cleaned DataFrame.
    """
    # locate the header row index
    header_idx = df.index[df.apply(lambda row: row.astype(str).str.contains(keyword).any(), axis=1)]
    
    if len(header_idx) == 0:
        # no header row found — return unchanged
        return df
    
    header_idx = header_idx[0]

    # set the header
    new_header = df.iloc[header_idx]
    df = df[header_idx + 1 :]  # drop the header row and all above it
    df.columns = new_header

    return df.reset_index(drop=True)


for link in links:
    response = requests.get(link)

    df = pd.read_excel(
        io.BytesIO(response.content),
        engine='openpyxl',
        header=None,                    
        sheet_name='Average Weekday'
    )

    df = set_header_row(df)

    # Add extracted date
    df["date"] = extract_date_from_link(link)            

    ridership_df = pd.concat([ridership_df, df], ignore_index=True)


In [266]:
# https://www.bart.gov/sites/default/files/docs/station-names.xls

station_map_df = pd.DataFrame({
    "code": [
        "RM","EN","EP","NB","BK","AS","MA","19","12","LM","FV","CL","SL","BF","HY","SH",
        "UC","FM","CN","PH","WC","LF","OR","RR","OW","EM","MT","PL","CC","16","24","GP",
        "BP","DC","CM","CV","ED","NC","WP","SS","SB","SO","MB","WD","OA","WS","AN","PC",
        "ML","BE"
    ],
    "stop_name": [
        "Richmond","El Cerrito Del Norte","El Cerrito Plaza","North Berkeley",
        "Downtown Berkeley","Ashby","MacArthur","19th Street Oakland",
        "12th Street / Oakland City Center","Lake Merritt","Fruitvale","Coliseum - OAC",
        "San Leandro","Bay Fair","Hayward","South Hayward","Union City","Fremont",
        "Concord","Pleasant Hill / Contra Costa Centre","Walnut Creek","Lafayette","Orinda","Rockridge",
        "West Oakland","Embarcadero","Montgomery Street","Powell Street",
        "Civic Center / UN Plaza","16th Street / Mission","24th Street / Mission","Glen Park",
        "Balboa Park","Daly City","Colma","Castro Valley","Dublin / Pleasanton",
        "North Concord / Martinez","Pittsburg / Bay Point","South San Francisco","San Bruno",
        "San Francisco International Airport","Millbrae (Caltrain Transfer Platform)","West Dublin / Pleasanton",
        "Oakland International Airport Station","Warm Springs / South Fremont","Antioch","Pittsburg Center",
        "Milpitas","Berryessa / North San Jose"
    ]
})

In [267]:
GTFS_URL = "https://www.bart.gov/dev/schedules/google_transit.zip"

response = requests.get(GTFS_URL)
z = zipfile.ZipFile(io.BytesIO(response.content))

# Load GTFS components into DataFrames
routes = pd.read_csv(z.open("routes.txt"))
stops = pd.read_csv(z.open("stops.txt"))
trips = pd.read_csv(z.open("trips.txt"), dtype={"route_id": str})
stop_times = pd.read_csv(z.open("stop_times.txt"))
calendar = pd.read_csv(z.open("calendar.txt"))

# Join trips with routes
trip_routes = trips.merge(routes, on="route_id", how="left")

# Join stop_times with stops
times_with_stops = stop_times.merge(stops, on="stop_id", how="left")

# Final joined table
full_schedule = (
    trip_routes
    .merge(times_with_stops, on="trip_id", how="left")
    .sort_values(["route_id", "trip_id", "stop_sequence"])
)

full_schedule['stop_name'] = full_schedule['stop_name'].str.replace("", "", regex=False)

mapped_schedule = full_schedule.merge(station_map_df, left_on="stop_name", right_on="stop_name", how="left")

# mapped_schedule.to_excel("C:/Users/chhri/Downloads/bart_full_schedule.xlsx")

In [268]:
df = ridership_df.rename(columns={'Exit Station Two-Letter Code': 'from_stop'})

# 2. Melt all OD columns into long form
df_long = df.melt(
    id_vars=['date', 'from_stop'],     # keep these as identifiers
    var_name='to_stop',                # new column representing destination
    value_name='ridership'             # ridership value
)

# 3. Drop rows where to_stop is NaN (your last row has a NaN column)
df_long = df_long.dropna(subset=['to_stop'])

# Optional: sort nicely
df_long = df_long.sort_values(['date', 'from_stop', 'to_stop']).reset_index(drop=True)

df_avg = (
    df_long
    .groupby(['from_stop', 'to_stop'], as_index=False)['ridership']
    .mean()
    .rename(columns={'ridership': 'avg_ridership'})
)

# Round up avg ridership to nearest integer and handle missing values
df_avg['avg_ridership'] = df_avg['avg_ridership'].apply(lambda x: int(math.ceil(x)) if pd.notna(x) else 0)

df_avg.head()

,from_stop,to_stop,avg_ridership
0,12,12,37
1,12,16,182
2,12,19,23
3,12,24,144
4,12,AN,91


In [277]:
stops_df = mapped_schedule.copy()

def parse_gtfs_time(t):
    """
    Parse time strings like '06:11:00' or '24:05:00' into minutes since midnight.
    GTFS allows hours >= 24, meaning times past midnight but same service day.
    """
    if pd.isna(t):
        return np.nan
    hh, mm, ss = map(int, str(t).split(":"))
    return hh * 60 + mm + ss / 60.0   # minutes since midnight (possibly > 24*60)

def compute_travel_times(stops_df):
    df = stops_df.copy()

    # parse times into numeric minutes
    df["arr_min"] = df["arrival_time"].apply(parse_gtfs_time)
    df["dep_min"] = df["departure_time"].apply(parse_gtfs_time)

    # sort properly
    df = df.sort_values(["route_id", "trip_id", "stop_sequence"])

    arc_rows = []

    for (route_id, trip_id), group in df.groupby(["route_id", "trip_id"]):
        group = group.sort_values("stop_sequence")

        dep_times = group["dep_min"].values
        arr_times = group["arr_min"].values
        seq      = group["stop_sequence"].values
        codes    = group["code"].values
        dep_str  = group["departure_time"].values
        arr_str  = group["arrival_time"].values

        for i in range(len(group) - 1):
            travel = arr_times[i+1] - dep_times[i]   # minutes
            travel_min = int(round(travel))

            arc_rows.append({
                "route_id": route_id,
                "trip_id": trip_id,
                "from_stop": codes[i],
                "to_stop": codes[i+1],
                "from_seq": seq[i],
                "to_seq": seq[i+1],
                "dep_time_str": dep_str[i],
                "arr_time_str": arr_str[i+1],
                "travel_time_min": travel_min,
                "arr_time": arr_times[i+1],
                "dep_time": dep_times[i]
            })
    df = pd.DataFrame(arc_rows)

    arc_rows = df.merge(df_avg, how='left', on=['from_stop','to_stop'])

    return pd.DataFrame(arc_rows)

arc_df = compute_travel_times(stops_df)
arc_df['from_stop_to_stop'] = arc_df['from_stop'] + "_" + arc_df['to_stop']


In [ ]:
V = arc_df['from_stop'] + "_" + arc_df['dep_time'] + "_" + arc_df['to_stop'] + "_" + arc_df['arr_time']

# --- Clean arc list ---
A = [a for a in arc_df['from_stop_to_stop'].unique() if isinstance(a, str)]

# --- Build station set ---
V = sorted(set(s for a in A for s in a.split("__")))

# --- Stage arcs (seq+1 arcs) ---
A_stage = [
    row['from_stop_to_stop']
    for _, row in arc_df.iterrows()
    if isinstance(row['from_stop_to_stop'], str)
       and row['to_seq'] == row['from_seq'] + 1
]
A_stage = sorted(set(A_stage))

# --- Overnight arcs (arrival next day) ---
A_ov = [
    row['from_stop_to_stop']
    for _, row in arc_df.iterrows()
    if isinstance(row['from_stop_to_stop'], str)
       and row['arr_time'] < row['dep_time']
]
A_ov = sorted(set(A_ov))

# --- Demand dictionary: d[a] = avg_ridership ---
d = {
    row['from_stop_to_stop']: row['avg_ridership']
    for _, row in arc_df.iterrows()
    if isinstance(row['from_stop_to_stop'], str)
}


In [314]:
def build_train_fleet_model(V, A, A_stage, A_ov, d, m, units_per_train):
    """
    Generic implementation of the model described in the screenshot.

    Inputs:
        V  : set of station-time nodes (list or iterable)
        A  : set of all arcs (list of arc IDs)
        A_stage : subset of A representing train stage arcs
        A_ov    : subset of A representing overnight arcs
        d  : dict mapping stage arc a -> required number of seats d[a]
        m  : max seats per train (capacity constraint)
        units_per_train : n in screenshot (seats per unit)
    """

    # Create model
    model = Model("Train_Fleet_Optimization")

    # -----------------------------
    # Decision Variables
    # -----------------------------
    # t[a] = number of train units assigned to arc a
    t = model.addVars(A, vtype=GRB.INTEGER, lb=0, name="t")

    # -----------------------------
    # Objective
    # -----------------------------
    # Minimize total units assigned on overnight arcs
    model.setObjective(quicksum(t[a] for a in A_ov), GRB.MINIMIZE)

    # -----------------------------
    # Constraints
    # -----------------------------
    # (1) Flow Conservation
    # For each station-time node v:
    #   sum outgoing = sum incoming
    for v in V:
        outgoing = [a for a in A if a.startswith(f"{v.split('_')[0]}_")]
        incoming = [a for a in A if a.endswith(f"_{v.split('_')[1]}")]
        model.addConstr(quicksum(t[a] for a in outgoing) ==
                        quicksum(t[a] for a in incoming),
                        name=f"flow_{v}")

    # (2) Capacity Constraint
    # t[a] ≤ m for each stage arc a ∈ A_stage
    for a in A_stage:
        model.addConstr(t[a] <= m, name=f"capacity_{a}")

    # (3) Demand Constraint
    # n * t[a] ≥ d[a]
    for a in A_stage:
        model.addConstr(m * t[a] >= d[a]/54, name=f"demand_{a}")

    # (4) Integrality is already handled by variable type

    return model.optimize()

build_train_fleet_model(
    V,
    A,
    A_stage,
    A_ov,
    d,
    6,
    54
)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 290 rows, 98 columns and 410 nonzeros
Model fingerprint: 0x2bd96400
Variable types: 0 continuous, 98 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [318]:
def build_train_fleet_model(V, A, A_stage, A_ov, d, m, units_per_train):
    """
    Generic implementation of the model described in the screenshot.

    Inputs:
        V  : set of station-time nodes (list or iterable)
        A  : set of all arcs (list of arc IDs)
        A_stage : subset of A representing train stage arcs
        A_ov    : subset of A representing overnight arcs
        d  : dict mapping stage arc a -> required number of seats d[a]
        m  : max seats per train (capacity constraint)
        units_per_train : n in screenshot (seats per unit)
    """

    # Create model
    model = Model("Train_Fleet_Optimization")

    # -----------------------------
    # Decision Variables
    # -----------------------------
    # t[a] = number of train units assigned to arc a
    t = model.addVars(A, vtype=GRB.INTEGER, lb=0, name="t")

    # -----------------------------
    # Objective
    # -----------------------------
    # Minimize total units assigned on overnight arcs
    model.setObjective(quicksum(t[a] for a in A_ov), GRB.MINIMIZE)

    # -----------------------------
    # Constraints
    # -----------------------------
    # (1) Flow Conservation
    # For each station-time node v:
    #   sum outgoing = sum incoming
    for v in V:
        outgoing = [a for a in A if a.startswith(f"{v.split('_')[0]}_")]
        incoming = [a for a in A if a.endswith(f"_{v.split('_')[1]}")]
        model.addConstr(quicksum(t[a] for a in outgoing) ==
                        quicksum(t[a] for a in incoming),
                        name=f"flow_{v}")

    # (2) Capacity Constraint
    # t[a] ≤ m for each stage arc a ∈ A_stage
    for a in A_stage:
        model.addConstr(t[a] <= m, name=f"capacity_{a}")

    # (3) Demand Constraint
    # n * t[a] ≥ d[a]
    for a in A_stage:
        model.addConstr(m * t[a] >= d[a]/54, name=f"demand_{a}")

    # (4) Integrality is already handled by variable type

    return model.optimize()

build_train_fleet_model(
    V,
    A,
    A_stage,
    A_ov,
    d,
    6,
    54
)


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700K, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 290 rows, 98 columns and 410 nonzeros
Model fingerprint: 0x2bd96400
Variable types: 0 continuous, 98 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [319]:
A

['AN_PC',
 'PC_WP',
 'WP_NC',
 'NC_CN',
 'CN_PH',
 'PH_WC',
 'WC_LF',
 'LF_OR',
 'OR_RR',
 'RR_MA',
 'MA_19',
 '19_12',
 '12_OW',
 'OW_EM',
 'EM_MT',
 'MT_PL',
 'PL_CC',
 'CC_16',
 '16_24',
 '24_GP',
 'GP_BP',
 'BP_DC',
 'DC_CM',
 'CM_SS',
 'SS_SB',
 'SB_SO',
 'SO_SO',
 'SO_MB',
 'SB_MB',
 'ED_WD',
 'WD_CV',
 'CV_BF',
 'BF_SL',
 'FV_LM',
 'LM_OW',
 'DC_BP',
 'BP_GP',
 'GP_24',
 '24_16',
 '16_CC',
 'CC_PL',
 'PL_MT',
 'MT_EM',
 'EM_OW',
 'OW_LM',
 'LM_FV',
 'SL_BF',
 'BF_CV',
 'CV_WD',
 'WD_ED',
 'OA_CL',
 'MB_SO',
 'SO_SB',
 'SB_SS',
 'SS_CM',
 'CM_DC',
 'OW_12',
 '12_19',
 '19_MA',
 'MA_RR',
 'RR_OR',
 'OR_LF',
 'LF_WC',
 'WC_PH',
 'PH_CN',
 'CN_NC',
 'NC_WP',
 'WP_PC',
 'PC_AN',
 'CL_OA',
 'SH_HY',
 'HY_BF',
 'LM_12',
 'MA_AS',
 'AS_BK',
 'BK_NB',
 'NB_EP',
 'EP_EN',
 'EN_RM',
 'FM_UC',
 'UC_SH',
 'BE_ML',
 'ML_WS',
 'WS_FM',
 'RM_EN',
 'EN_EP',
 'EP_NB',
 'NB_BK',
 'BK_AS',
 'AS_MA',
 '12_LM',
 'BF_HY',
 'HY_SH',
 'SH_UC',
 'UC_FM',
 'FM_WS',
 'WS_ML',
 'ML_BE']